# LETS GET THIS STARTED

In [2]:
import pandas as pd 

In [10]:
#renamed the files to make it more accessible and intuitive.
#reading csv files into pd dataframes for manipulation and data wrangling
cardholder_acc = pd.read_csv('../csvs_datathon/accy_dim.csv')
statement_data = pd.read_csv('../csvs_datathon/statement_fact.csv')
transaction_data = pd.read_csv('../csvs_datathon/transaction_fact.csv')
wrld_transaction_data = pd.read_csv('../csvs_datathon/wrld_stor_tran_fact.csv')
customer_id = pd.read_csv('../csvs_datathon/syf_id.csv')
acc_lvl_features = pd.read_csv('../csvs_datathon/rams_batch_cur.csv')
fraud_claim_case = pd.read_csv('../csvs_datathon/fraud_claim_case.csv')
fraud_claim_tran = pd.read_csv('../csvs_datathon/fraud_claim_tran.csv')